In [2]:
import pandas as pd
import numpy as np
import pyterrier as pt
import os, shutil

wit = pd.read_csv("wit_1_percent.tsv", sep="\t")
# Filter only English
wit = wit[wit["language"] == "en"].reset_index(drop=True)

In [3]:
wit['docno'] = [str(i) for i in wit.index]
# text_feature_cols = ['page_title', 'section_title', 'hierarchical_section_title','caption_reference_description', 'caption_attribution_description', 'context_page_description', 'context_section_description']
text_feature_cols = ['page_title', 'section_title', 'caption_attribution_description', 'context_section_description']
wit['text_features'] = wit[text_feature_cols].apply(
    lambda x: ', '.join(x.dropna().astype(str)),
    axis=1
)
wit.head()

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description,docno,text_features
0,en,"https://en.wikipedia.org/wiki/Chinatown,_Sydney",http://upload.wikimedia.org/wikipedia/commons/...,"Chinatown, Sydney",Bilingual street signs,"Chinatown, Sydney / Bilingual street signs",NaN,English: Hay Street 中文（繁體）‎: 禧街,NaN,image/jpeg,960,1280,False,False,True,Chinatown is an urban enclave situated in the ...,There are many bilingual street signs across C...,0,"Chinatown, Sydney, Bilingual street signs, Eng..."
1,en,https://en.wikipedia.org/wiki/Jayson_Musson,https://upload.wikimedia.org/wikipedia/commons...,Jayson Musson,NaN,Jayson Musson,"Jayson Musson on October 29, 2007","English: Jayson Scott Musson on October 29, 2007",NaN,image/jpeg,400,300,False,True,False,Jayson Scott Musson is an artist who lives and...,Jayson Scott Musson is an artist who lives and...,1,"Jayson Musson, English: Jayson Scott Musson on..."
2,en,https://en.wikipedia.org/wiki/Euodynerus_megaera,https://upload.wikimedia.org/wikipedia/commons...,Euodynerus megaera,NaN,Euodynerus megaera,NaN,English: Euodynerus megaera,NaN,image/jpeg,433,531,True,True,False,Euodynerus megaera is a species of stinging wa...,Euodynerus megaera is a species of stinging wa...,2,"Euodynerus megaera, English: Euodynerus megaer..."
3,en,https://en.wikipedia.org/wiki/Simon_W._Rosendale,https://upload.wikimedia.org/wikipedia/commons...,Simon W. Rosendale,NaN,Simon W. Rosendale,NaN,"English: Simon Wolfe Rosendale (June 23, 1842 ...",NaN,image/jpeg,727,404,True,True,False,Simon Wolfe Rosendale was an American lawyer a...,"Simon Wolfe Rosendale (June 23, 1842 – April 2...",3,"Simon W. Rosendale, English: Simon Wolfe Rosen..."
4,en,https://en.wikipedia.org/wiki/List_of_National...,https://upload.wikimedia.org/wikipedia/commons...,List of National Natural Landmarks in Florida,National Natural Landmarks,List of National Natural Landmarks in Florida ...,NaN,English: Photograph of Rainbow Springs in Mari...,NaN,image/jpeg,1536,2048,False,True,False,The National Natural Landmarks in Florida incl...,NaN,4,"List of National Natural Landmarks in Florida,..."


In [3]:
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-15.0.2"
if not pt.started():
    pt.init(boot_packages=["io.anserini:anserini:0.9.2:fatjar"])

shutil.rmtree("wit_index")
pd_indexer = pt.DFIndexer("C:\\Users\\jovin\\Documents\\Computer Science (FR)\\Year 4\\Level 4 Individual Project\\Automatic-Illustration-of-Text\\WIT\\wit_index", tokeniser="UTFTokeniser")
indexref = pd_indexer.index(wit['text_features'], wit['docno'])

PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [6]:
index = pt.IndexFactory.of("C:\\Users\\jovin\\Documents\\Computer Science (FR)\\Year 4\\Level 4 Individual Project\\Automatic-Illustration-of-Text\\WIT\\wit_index\\data.properties")
topics = pd.DataFrame([["1", "horse"]], columns=['qid', 'query'])
retr = pt.BatchRetrieve(index, wmodel='TF_IDF', num_results=1000)
res = retr.transform(topics)

print(res)

# Get top 5 dog pics
documents = wit[wit['docno'].isin(res['docno'][0:10])]
url = documents['image_url'].values
print(url)
print([str(url) for url in documents["page_url"].values])

    qid  docid  docno  rank     score  query
0     1  24389  24389     0  6.996689  horse
1     1  49954  49954     1  6.850152  horse
2     1  44461  44461     2  6.830736  horse
3     1    900    900     3  6.803589  horse
4     1  30789  30789     4  6.799042  horse
..   ..    ...    ...   ...       ...    ...
669   1   8226   8226   669  1.338867  horse
670   1  31070  31070   670  1.319480  horse
671   1  16668  16668   671  1.315670  horse
672   1  23508  23508   672  1.235405  horse
673   1  33879  33879   673  0.772500  horse

[674 rows x 6 columns]
['https://upload.wikimedia.org/wikipedia/commons/e/ec/Hodsons_Horse_France_1917_IWM_Q_2061.jpg'
 'https://upload.wikimedia.org/wikipedia/commons/e/e2/Pony_express_crop.jpg'
 'http://upload.wikimedia.org/wikipedia/commons/2/28/2nd_Horse_Grenadier.jpg'
 'https://upload.wikimedia.org/wikipedia/commons/3/38/Naadam_rider_Mongolia.jpg'
 'https://upload.wikimedia.org/wikipedia/commons/f/fb/The_Farriers_Arms_-_geograph.org.uk_-_527610.jpg'


In [7]:
import requests

headers ={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}

for i, image in enumerate(url):
    with open(f"images\\{i}.jpg", "wb") as image_file:
        image_file.write(requests.get(image, headers=headers).content)

In [8]:
documents['caption_attribution_description'].values

array(["English: Forward scouts of the 9th Hodson's Horse, an Indian cavalry regiment, pause to consult a map, near Vraignes, France.",
       nan,
       'English: Trooper of the 2nd Horse Grenadier Guards, by David Morier, c. 1750.',
       'Mongolia, Naadam rider',
       "English: The Farriers Arms Shilbottle's public house.",
       'English: Dutch Harness Horse stallion imported to the United States and exhibited at a barn tour in Scottsdale, Arizona',
       'Deutsch: Porträt des Infanten Don Diego (1575-1582)',
       '中文（台灣）\u200e: 打馬所用的打馬錢',
       'Cavalli della Madonna, Marstall, Kloster Einsiedeln in Einsiedeln (Switzerland)',
       'English: Horse drawn litter of a style used formerly as a battlefield ambulance, now as hippotherapy.Nederlands: huifbed_te_Heel'],
      dtype=object)

In [4]:
wit.size

1027349